# 🏥 Affinage (*fine-tuning*) de modèles encodeurs PARTAGES (BERT-like) pour  l'annotation (classification de tokens)

### Installation des libraries

In [ ]:
# Parfois il est plus simple d'installer torch via conda (ou d'utiliser le
# paquet pytorch). S'il y a des erreurs avec numpy installez 'numpy<2'.

# On utilise `datasets<4` car la version 4 de tokenizer ne permet plus de charger
# des jeux de données à l'aide de scripts de chargement. Certains jeux de
# données utilisent toujours des scripts de chargements (notamment DrBenchmark).

%pip install 'transformers[torch]' 'datasets<4' tensorboardX torch
%pip install seqeval evaluate

In [ ]:
# Petit hack pour éviter un avertissement spécifique à Google Colab sur le
# chargement du jeton huggingface
try:
  from huggingface_hub.utils import _auth
  _auth._get_token_from_google_colab = lambda: None
except ImportError:
  pass

In [ ]:
######################################
###########    A Modifier    #########
######################################

# Si le jeu de donnée, modèle ou tokeniseur n'est pas public sur huggingface il
# faut utiliser un jeton de connexion à créer ici : https://huggingface.co/settings/tokens

# Pour l'instant les modèles PARTAGES sont privés et accessibles avec le jeton
#  communiqué avec ce fichier
HF_TOKEN = ""

# Ici on utilise un des modèles du projet PARTAGES
# A modifier pour utiliser un autre modèle comme base d'affinage
BASE_MODEL_NAME = "WP4/PARTAGES-encoder-v1"

######################################
assert HF_TOKEN != ""

### Chargement du tokeniseur

In [ ]:
import os
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, token=HF_TOKEN)

### Chargement et préparation des données

Chargement des données au format ConLL-2003, la colonne 'tokens' est une liste qui représente chaque mot d'un document, la colonne 'ner_tags' est de la même longueur que 'tokens' et indique l'étiquette de chaque mot.

```python
[{'id': str, 'tokens': list(str), 'ner_tags': list(int)}, ...]
```

In [ ]:
def tokenize_and_align_labels(examples, tokens_column, labels_column, label_all_tokens=False, max_length=None):
    """Retokenize each word to match current tokenizer and align new sub tokens with labels.
    Adapted from from https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=n9qywopnIrJH

    [("avec", 0), ("une", 0), ("tachycardie", 1), ("à", 0), ("115", 0)]

    With label_all_tokens = False
    [("avec", 0), ("une", 0), ("tac", 1), ("##hy", -100), ("##card", -100), ("##ie", -100), ("à", 0), ("11", 0), ("##5", -100)]

    With label_all_tokens = True
    [("avec", 0), ("une", 0), ("tac", 1), ("##hy", 1), ("##card", 1), ("##ie", 1), ("à", 0), ("11", 0), ("##5", 0)]
    """
    if max_length is None:
        max_length = tokenizer.model_max_length
    tokenized_inputs = tokenizer(
        list(examples[tokens_column]),
        truncation=True, max_length=max_length,
        padding='do_not_pad', is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples[labels_column]):
        label_ids = []
        previous_word_idx = None
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
#########################################################
###########    ZONE A Modifier POUR UTILISER    #########
###########         VOS PROPRES DONNEES         #########
##          ATTENTION AUX DONNEES PRIVEES NE PAS       ##
##              EXECUTER SUR LE CLOUD                  ##
#########################################################
# En sortie de ces cellule:
#  - label_list : liste des différentes étiquettes
#  - train_dataset : un Dataset tokenisé avec une colonne "label"
#  - eval_dataset : un Dataset tokenisé avec une colonne "label"

# L'exemple est avec le corpus DEFT2021 que l'on peut obtenir via la collection
#  de DrBenchmark (cf github.com/DrBenchmark/DrBenchmark)
# https://aclanthology.org/2024.lrec-main.478.pdf
# Chaque cas clinique est annoté en extraction d'information au niveau token.
#  Après extraction, le corps du texte est tokenisé via la fonction
#  preprocess_function.

import datasets

dataset = datasets.load_dataset('DrBenchmark/DEFT2021', 'ner', trust_remote_code=True)
dataset

In [ ]:
label_list = dataset['train'].features[f"ner_tags"].feature.names

# Si les features du dataset ne sont pas définies
#label_list = list({l for d in dataset['train']['ner_tags'] for l in d})

# Si la colonne des etiquettes contient des chaines de caractère il faut les transformer
#  en entier
"""id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in enumerate(label_list)}
dataset = dataset.map(lambda d: {'ner_tags': [label2id[l] for l in d['ner_tags']]})"""

In [ ]:
# Tokenize dataset
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    fn_kwargs={
        'tokens_column': 'tokens',
        'labels_column': 'ner_tags',
        'label_all_tokens': False,
        # On met un maximum a 8192 pour éviter les erreurs de mémoire liés au
        #  traitement de long documents mais c'est a modifier selon les besoins
        #  et capacités de calcul. Certains modèle n'ayant pas de limite.
        'max_length': min(tokenizer.model_max_length, 8192),
    },
    batched=True, batch_size=100,
)

# Split dataset into train and validation sets
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

#############################################

In [ ]:
import random
import textwrap

import numpy as np

# Affichage d'exemples au hasard
to_show = [15, random.randint(0, len(train_dataset))]

print(f"Etiquettes : {label_list}")
idx_train_longest = int(np.argmax([len(e) for e in train_dataset['input_ids']]))
print(f"Plus long  doc train : idx {idx_train_longest:4d}, len {len(train_dataset[idx_train_longest]['input_ids']):4d}")
idx_train_shortest = int(np.argmin([len(e) for e in train_dataset['input_ids']]))
print(f"Plus court doc train : idx {idx_train_shortest:4d}, len {len(train_dataset[idx_train_shortest]['input_ids']):4d}")

idx_eval_longest = int(np.argmax([len(e) for e in eval_dataset['input_ids']]))
print(f"Plus long  doc valid : idx {idx_eval_longest:4d}, len {len(eval_dataset[idx_eval_longest]['input_ids']):4d}")
idx_eval_shortest = int(np.argmin([len(e) for e in eval_dataset['input_ids']]))
print(f"Plus court doc valid : idx {idx_eval_shortest:4d}, len {len(eval_dataset[idx_eval_shortest]['input_ids']):4d}")

for i in to_show:
    doc = train_dataset[i]
    print(f"------ Document train {i} ------")
    print(f"Nb. tokens: {len(doc['input_ids'])}")
    tokens = ' '.join(tokenizer.convert_ids_to_tokens(doc['input_ids'])).replace('\n', '\\n')
    print(f"Tokens: {'\n'.join(textwrap.wrap(tokens, width=120, break_long_words=False, break_on_hyphens=False))}")
    print(f"Etiquettes: {doc['labels']}")

### Définition des métriques a rapporter

In [ ]:
import importlib

import evaluate
from seqeval.metrics import accuracy_score, classification_report

class Seqeval(evaluate.Metric):
    def _info(self):
        return evaluate.MetricInfo(
            description="", citation="",
            homepage="https://github.com/chakki-works/seqeval",
            features=datasets.Features({
                "predictions": datasets.Sequence(datasets.Value("string", id="label"), id="sequence"),
                 "references": datasets.Sequence(datasets.Value("string", id="label"), id="sequence")
            }))

    def _compute(
        self,
        predictions,
        references,
        suffix: bool = False,
        scheme: str | None = None,
        mode: str | None = None,
        sample_weight: list[int] | None = None,
        zero_division: str | int = "warn",
    ):
        if scheme is not None:
            try:
                scheme_module = importlib.import_module("seqeval.scheme")
                scheme = getattr(scheme_module, scheme)
            except AttributeError:
                raise ValueError(f"Scheme should be one of [IOB1, IOB2, IOE1, IOE2, IOBES, BILOU], got {scheme}")
        report = classification_report(
            y_true=references,
            y_pred=predictions,
            suffix=suffix,
            output_dict=True,
            scheme=scheme,
            mode=mode,
            sample_weight=sample_weight,
            zero_division=zero_division,
        )
        report.pop("macro avg")
        report.pop("weighted avg")
        overall_score = report.pop("micro avg")

        scores = {
            type_name: {
                "precision": score["precision"],
                "recall": score["recall"],
                "f1": score["f1-score"],
                "number": score["support"],
            }
            for type_name, score in report.items()
        }
        scores["overall_precision"] = overall_score["precision"]
        scores["overall_recall"] = overall_score["recall"]
        scores["overall_f1"] = overall_score["f1-score"]
        scores["overall_accuracy"] = accuracy_score(y_true=references, y_pred=predictions)
        return scores

In [ ]:
import numpy as np

# On utilise la metrique SeqEval pour évaluer l'annotation de tokens
metric = Seqeval()

def compute_metrics(p):
    global metric
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=.0)

    macro_values = [results[r]["f1"] for r in results if "overall_" not in r]
    macro_f1 = sum(macro_values) / len(macro_values)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"], "macro_f1": macro_f1,
        "accuracy": results["overall_accuracy"]
    }

### Chargement du modèle de base


Maintenant que l'on a chargé les données on connait le nombre d'étiquettes donc
on peut charger le modèle et initialiser sa couche de classification.

In [ ]:
#############################################
############    A Modifier    ###############
#############################################

SEED = 42  # Pour faciliter la reproductibilité des résultats

#############################################

In [ ]:
from transformers import set_seed
from transformers import AutoModelForTokenClassification

# On initialise le générateur de nombres aléatoires avant de charger le modèle
# pour s'assurer que l'initialisation des poids de la couche de classification
# soient les mêmes à chaque fois.
set_seed(SEED)
model = AutoModelForTokenClassification.from_pretrained(
    BASE_MODEL_NAME, token=HF_TOKEN, num_labels=len(label_list),
    # Pour avoir de belles etiquettes en sortie du modèle
    id2label = {i: l for i, l in enumerate(label_list)},
    label2id = {l: i for i, l in enumerate(label_list)},
)
print(model)

### Préparation de l'entrainement

In [ ]:
#############################################
############    A Modifier    ###############
#############################################

OUTPUT_MODEL_NAME = "PARTAGES-encoder-v1-annot"
OUTPUT_DIR = os.path.join(os.environ['HOME'], 'PARTAGES', OUTPUT_MODEL_NAME)

#############################################

Le résultat de l'entrainement sera stocké dans:
```
~/PARTAGES/PARTAGES-encoder-v1-annot/
- checkpoints/
  - checkpoint-10
  - checkpoint-20
  - checkpoint-30
  - checkpoint-...
- training_args.json
- model.pt
- tokenizer.
```

Ainsi les points de contrôles sont conservés, et il sera possible de charger le meilleur modèle résultant de l'entrainement en écrivant:

```python
from transformers import AutoTokenizer, AutoModelForTokenClassification
model_path = os.path.join(os.environ['HOME'], "PARTAGES", "PARTAGES-encoder-v1-annot")
# /!\ Il est important d'utiliser AutoModelForTokenClassification et non
# AutoModel (qui ne va pas charger la couche de classification)
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Pour charger un checkpoint spécifique
model = AutoModelForTokenClassification.from_pretrained(model_path + "/checkpoints/checkpoint-20")
tokenizer = AutoTokenizer.from_pretrained(model_path)
```

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer)

# Pour plus d'arguments
# https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments

training_args = TrainingArguments(
    os.path.join(OUTPUT_DIR, 'checkpoints'),
    overwrite_output_dir=False,  # pour continuer l'entrainement
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2.0e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    metric_for_best_model="macro_f1",
    load_best_model_at_end=True,
    greater_is_better=True,
    logging_strategy="epoch",
    save_only_model=True,
    save_total_limit=5,
    seed=42,
    report_to='tensorboard',
)

# Pour augmenter le batch_size mais ne pas avoir d'erreur de mémoire il est possible
#  d'utiliser le paramètre `gradient_accumulation_steps` qui attend N batchs avant
#  d'effectuer la rétropropagation. Par exemple `batch_size=64, gradient_accumulation_steps=1`
#  est équivalent à `batch_size=32, gradient_accumulation_steps=2` ou
#  `batch_size=16, gradient_accumulation_steps=4`
if os.path.exists(OUTPUT_DIR):
    print(f"""Le dossier de sortie existe déjà !
L'entrainement va continuer s'il a été arrêté ou ne rien faire si l'entrainement a terminé.
Chemin : {OUTPUT_DIR}
Soyez sûr de ce qu'il se passe ou bien supprimez le dossier.""")

In [ ]:
# Exemple de document passé par le DataCollator (ajout de padding pour que tout
#  les documents d'un même lot (batch) aient la même longueur
sampled_batch = train_dataset.select_columns(['input_ids', 'attention_mask', 'labels']).select(range(8))
collated_batch = data_collator(sampled_batch)
collated_doc = collated_batch['input_ids'][0]

print("Exemple de document passé par le Data Collator (ce que le modèle va voir):")
doc_to_print = ' '.join(tokenizer.convert_ids_to_tokens(collated_doc)).replace('\n', '\\n')
print('\n'.join(textwrap.wrap(doc_to_print, width=120, break_long_words=False, break_on_hyphens=False)))

In [ ]:
# Supprimer ou mettre à `False` pour un entrainement réel
debugging_mode = True
if debugging_mode:
    n_samples = min(len(train_dataset), len(eval_dataset))
    n_samples = min(n_samples, 20)
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("!!!            Mode de debug          !!!")
    print(f"!!!  Seuls {n_samples} examples sont utilisés  !!!")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    train_dataset = train_dataset.select(range(n_samples))
    eval_dataset = eval_dataset.select(range(n_samples))

In [ ]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
    # Avec EarlyStopping, l'entrainement va s'arrêter après 3 epoch si
    #  la macro_f1 (`metric_for_best_model`) ne s'améliore pas
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

In [ ]:
import json
trainer.save_model(os.path.join(OUTPUT_DIR))
with open(os.path.join(OUTPUT_DIR, "training_args.json"), 'w') as f:
  json.dump(json.loads(training_args.to_json_string()) | {'basemodel_name_or_path': model.name_or_path}, f)

## Utilisation du modèle entrainé

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
model_path = os.path.join(os.environ['HOME'], "PARTAGES", OUTPUT_MODEL_NAME)
# /!\ Il est important d'utiliser AutoModelForTokenClassification et non
# AutoModel (qui ne va pas charger la couche de classification)

In [ ]:
from transformers import pipeline
print(f"Chargement du modèle {OUTPUT_MODEL_NAME}.")
pipe = pipeline("token-classification", model=model_path, device="cpu")

In [ ]:
def preprocessing(text):
    # Si un pré-traitement a été effectué pour créer le corpus d'entrainement
    #  il faut reproduire ce prétraitement ici.
    return text.replace("c'est", "c' est")

texts = [
    "La patiente est enrhummée, et a pris du paracétamol.",
    "Une deuxième phrase."
]
texts = [preprocessing(t) for t in texts]

# Pour le paramètre `aggregation_strategy` voir https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TokenClassificationPipeline.aggregation_strategy
predictions = pipe(
    texts,
    batch_size=2,
    # Utilise le premier sous-token de chaque mot comme étiquette
    aggregation_strategy="first"
)

In [ ]:
# Note: l'etiquette sosy signifie "Signes Ou SYmptomes"
for t, p in zip(texts, predictions):
    print(f"Le modèle a identifié les entités suivantes:")
    print(p)
    print(f"dans le document: {t[:50]} [...]")
    print()


## Hyperparameter tuning

Suivant le temps de calcul à votre disposition et de la taille de vos jeux de données. Il est possible de chercher de meilleurs hyperparamètres automatiquement à l'aide de la bibliothèque `optuna`.

In [ ]:
%pip install optuna

On choisit ici de chercher de meilleures valeurs pour la `learning_rate`, le `weight_decay` et une meilleure `seed` qui initialise les paramètres.

In [ ]:
from optuna import Trial

def hp_space(trial: Trial) -> dict[str, float | int | str]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-6, 2e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.005, 0.1, log=True),
        "seed": trial.suggest_int("seed", 1, 40),
    }

# Utilisation d'une fonction model_init pour s'assurer de la reproducibilité
def model_init():
    global label_list, base_model_name
    return AutoModelForTokenClassification.from_pretrained(
        base_model_name, token=HF_TOKEN, num_labels=len(label_list),
        # Pour avoir de belles etiquettes en sortie du modèle
        id2label = {i: l for i, l in enumerate(label_list)},
        label2id = {l: i for i, l in enumerate(label_list)},
    )

# modification du trainer déjà initialisé avant pour bien ré-initialiser le
#  modèle à chaque essai
trainer.model_init = model_init

In [ ]:
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, n_trials=3)

In [ ]:
best_run